# RNN cross validation
Classify pc vs nc RNA.
Use K=2,3,4.

Set aside the 20% test set, stratified by length.
On the remaining 80%,
perform 5-fold cross validation.

Test subsets of the data with RNN.
To do: Read sequences (not K-mers).

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import ShuffleSplit


# For cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# For the manual cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

tf.keras.backend.set_floatx('float32')

In [2]:
def read_features(nc_file,pc_file):
    nc = pd.read_csv (nc_file)
    pc = pd.read_csv (pc_file)
    nc['class']=0
    pc['class']=1
    rna_mer=pd.concat((nc,pc),axis=0)
    return rna_mer

# Split into train/test stratified by sequence length.
def sizebin(df):
    return pd.cut(df["seqlen"],
                              bins=[0,1000,2000,4000,8000,16000,np.inf],
                              labels=[0,1,2,3,4,5])
def make_train_test(data):
    bin_labels= sizebin(data)
    from sklearn.model_selection import StratifiedShuffleSplit
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=37863)
    # split(x,y) expects that y is the labels. 
    # Trick: Instead of y, give it it the bin labels that we generated.
    for train_index,test_index in splitter.split(data,bin_labels):
        train_set = rna_mer.iloc[train_index]
        test_set = rna_mer.iloc[test_index]
    return (train_set,test_set)

def prepare_test_set(test_set):
    y_test=   test_set[['class']].copy()
    X_test=   test_set.div(test_set['seqlen'],axis=0)
    X_test=   X_test.drop(columns=['class','seqnum','seqlen'])
    return (X_test,y_test)

def prepare_train_set(train_set):
    y_train_all=  train_set[['class']].copy()
    X_train_all=  train_set.div(train_set['seqlen'],axis=0)
    X_train_all=  X_train_all.drop(columns=['class','seqnum','seqlen'])
    return (X_train_all,y_train_all)

def subset(data_set,min_len,max_len):
    print("original "+str(data_set.shape))
    too_short = data_set[ data_set['seqlen'] < min_len ].index
    data_set.drop(too_short,inplace=True)
    print("drop short "+str(data_set.shape))
    too_long = data_set[ data_set['seqlen'] >= max_len ].index
    data_set.drop(too_long,inplace=True)
    print("drop long "+str(data_set.shape))
    return data_set

def do_cross_validation(X_train_all,K):
    cv_scores = []
    act="sigmoid"
    dt='float32'
    fold=0
    eps=100
    splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=37863)
    for train_index,valid_index in splitter.split(X_train_all):
        X_train=X_train_all.iloc[train_index]
        y_train=y_train_all.iloc[train_index]
        X_valid=X_train_all.iloc[valid_index]
        y_valid=y_train_all.iloc[valid_index]
        mlp = keras.models.Sequential([
            keras.layers.LayerNormalization(trainable=False),
            keras.layers.Dense(32, activation=act,dtype=dt),
            keras.layers.Dense(32, activation=act,dtype=dt),
            keras.layers.Dense(1,  activation=act,dtype=dt)
        ])
        seq_len=None  # none indicates variable length
        input_features=4**K   # 64 DNA K-mers at K=3
        rnn2 = keras.models.Sequential([
            keras.layers.SimpleRNN(16, return_sequences=True, 
                                   input_shape=[seq_len,input_features]),
            keras.layers.SimpleRNN(16, return_sequences=True),
            keras.layers.SimpleRNN(16, return_sequences=True),
            keras.layers.SimpleRNN(1),
        ])

        bc=tf.keras.losses.BinaryCrossentropy(from_logits=False)
        rnn2.compile(loss=bc, optimizer="Adam",metrics=["accuracy"])
        history=rnn2.fit(X_train, y_train, # batch_size=10, default=32 works nicely
                epochs=eps, verbose=0,  # verbose=1 for ascii art, verbose=0 for none
                validation_data=(X_valid,y_valid) )
                        
        fold += 1
        print("Fold %d, %d epochs"%(fold,eps))

        pd.DataFrame(history.history).plot(figsize=(8,5))
        plt.grid(True)
        plt.gca().set_ylim(0,1)
        plt.show()

        scores = mlp.evaluate(X_valid, y_valid, verbose=0)
        print("%s: %.2f%%" % (mlp.metrics_names[1], scores[1]*100))
        cv_scores.append(scores[1] * 100)
    print()
    print("Validation core mean %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))


## Load K-mer frequencies, K=2

In [3]:
rna_mer = read_features('ncRNA.2mer.features.csv','pcRNA.2mer.features.csv')
rna_mer = subset(rna_mer,1000,2000)
(train_set,test_set)=make_train_test(rna_mer)
(X_test,y_test)=prepare_test_set(test_set)
(X_train_all,y_train_all)=prepare_train_set(train_set)
print(X_train_all.shape,y_train_all.shape)
do_cross_validation(X_train_all,2)

original (37863, 19)
drop short (9631, 19)
drop long (2907, 19)
(2325, 16) (2325, 1)


ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
  

## K-mer frequency, K=3

In [ ]:
rna_mer = read_features('ncRNA.3mer.features.csv','pcRNA.3mer.features.csv')
rna_mer = subset(rna_mer,1000,2000)
(train_set,test_set)=make_train_test(rna_mer)
(X_test,y_test)=prepare_test_set(test_set)
(X_train_all,y_train_all)=prepare_train_set(train_set)
print(X_train_all.shape,y_train_all.shape)
do_cross_validation(X_train_all,3)

## K-mer frequency, K=4

In [ ]:
rna_mer = read_features('ncRNA.4mer.features.csv','pcRNA.4mer.features.csv')
rna_mer = subset(rna_mer,1000,2000)
(train_set,test_set)=make_train_test(rna_mer)
(X_test,y_test)=prepare_test_set(test_set)
(X_train_all,y_train_all)=prepare_train_set(train_set)
print(X_train_all.shape,y_train_all.shape)
do_cross_validation(X_train_all,4)